In [1]:
import numpy as np  # for linear algebra
import pandas as pd # for data processing

In [2]:
# for EDA
import matplotlib.pyplot as plt
import seaborn as sns 

In [3]:
from sklearn.model_selection import train_test_split # for spliting training data and test data 
from sklearn.impute import SimpleImputer # for filling missing values
from sklearn.preprocessing import OneHotEncoder , MinMaxScaler # for dealing with categorical data
from sklearn.compose import ColumnTransformer  # for doing operations on the columns without separating them from dataframe
from sklearn.pipeline import Pipeline  # for making pipeline 
from sklearn.ensemble import RandomForestClassifier  # predicting model 

In [4]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.shape

(891, 12)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
# dropping unnecessary columns
df.drop( columns = ['PassengerId' , 'Ticket' , 'Cabin' , 'Name'] , inplace = True )

In [15]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [17]:
X_train , X_test , y_train , y_test  = train_test_split(df.drop(columns=['Survived']) ,
                                                        df['Survived'] , 
                                                        test_size = 0.2 , 
                                                        random_state = 42 )

In [19]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [21]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [23]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [53]:
# In this I am visualizing the order of  Dataframe after trf1
encoded_X_train = trf1.fit_transform(X_train)
feature_names = trf1.get_feature_names_out()  # will return the final feature names in order
encoded_df = pd.DataFrame(encoded_X_train, columns=feature_names)
encoded_df.head()

,impute_age__Age,impute_embarked__Embarked,remainder__Pclass,remainder__Sex,remainder__SibSp,remainder__Parch,remainder__Fare
0,45.5,S,1,male,0,0,28.5
1,23.0,S,2,male,0,0,13.0
2,32.0,S,3,male,0,0,7.925
3,26.0,S,3,male,1,0,7.8542
4,6.0,S,3,female,4,2,31.275


In [26]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [40]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False  , handle_unknown='ignore'),[1,3])
],remainder='passthrough')

In [43]:
trf3 = RandomForestClassifier(n_estimators=200, max_depth=7,random_state=42)

In [45]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3)
])

In [47]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('trf3',
                 RandomForestClassifier(max_depth=7, n_estimators=200,
                                        random_state=42))])

In [49]:
y_pred = pipe.predict(X_test)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8156424581005587